In [7]:
from collections import Counter
import numpy as np
import pandas as pd
import pickle

In [5]:
def processData(ticker):
    days = 7
    df = pd.read_csv('NASDAQ_JoinedCloses.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)

    for i in range(1, days + 1):
        df['{}_{}'.format(ticker, i)] = (df[ticker.shift(-i)] - df[ticker]) / df[ticker]

    df.fillna(0, inplace=True)
    return tickers, df

In [6]:
def actionDecision(*args): # *args allows taking any number of arguments
    cols = [c for c in args]
    required = 0.02 # stock price changing by 2%

    for col in cols:
        if col > required:
            return 1 # indicates buy position
        if col < -required:
            return -1 # indicates sell position
        return 0 # indicates hold position

In [ ]:
def extractFeatures(ticker):
    tickers, df = processData(ticker)
    df['{}_target'.format(ticker)] = list(map(actionDecision, 
    df['{}_1d'.format(ticker)], 
    df['{}_2d'.format(ticker)],
    df['{}_3d'.format(ticker)],
    df['{}_4d'.format(ticker)],
    df['{}_5d'.format(ticker)],
    df['{}_6d'.format(ticker)],
    df['{}_7d'.format(ticker)]
    ))

    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data Spread:', Counter(str_vals))
    df.fillna(0, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)

    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df.fillna(0, inplace=True)

    X = df_vals.values
    y = df['{}_target'.format(ticker)].values

    return X, y, df